### Install all dependencies 

```
pip install rasterio[s3] bs4 requests mapboxgl
```

In [1]:
import os
import json
import rasterio
import requests
import bs4 as BeautifulSoup

from concurrent import futures
from rasterio.warp import transform_bounds

from mapboxgl.utils import *
from mapboxgl.viz import *

In [5]:
url = 'https://www.digitalglobe.com/opendata/hurricane-florence/pre-event'

# Read Page
r = requests.get(url)

# Use BeautifulSoup to parse and extract all imagey links
soup = BeautifulSoup.BeautifulSoup(r.text)
s = soup.findAll('a',attrs={"class":"opendata__tilelinks"})

list_file = list(set([l.get('href') for l in s if not l.get('href').endswith('ovr')]))

In [6]:
len(list_file)

4343

### Create Footprint Geojson

In [ ]:
def worker(url):
    try:

        with rasterio.open(url) as src:
            wgs_bounds = transform_bounds(*[src.crs, 'epsg:4326'] + list(src.bounds), densify_pts=21)

        return {
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [wgs_bounds[0], wgs_bounds[3]],
                        [wgs_bounds[0], wgs_bounds[1]],
                        [wgs_bounds[2], wgs_bounds[1]],
                        [wgs_bounds[2], wgs_bounds[3]],
                        [wgs_bounds[0], wgs_bounds[3]]
                    ]
                ]
            },
            "properties": {
                "url": url,
            },
            "type": "Feature"
        }
    
    except:
        # Return empty dict on error
        return {}

    
with futures.ThreadPoolExecutor(max_workers=50) as executor:
    responses = executor.map(worker, list_file)

In [ ]:
# Filter only good responses 
features = [r for r in responses if r]
len(features)

In [ ]:
# Save geojson
with open('dg_pre_event.geojson', 'w') as f:
    f.write(json.dumps({'type': 'FeatureCollection', 'features': features}))

#### Visualize footprint

In [2]:
token = os.getenv('MAPBOX_ACCESS_TOKEN')

# Create the viz from the dataframe
viz = LinestringViz('dg_pre_event.geojson',
                access_token=token,
                height='400px',
                center = (-95, 40),
                zoom = 3,
                below_layer = 'waterway-label'
              )
viz.show()